# Fator de Visão de Céu a partir do LiDAR

Podemos também determinar o Fator de Visão de céu, ou Sky View Factor diretamente do LiDAR sem gerar um raster.
Para fins de performance podemos reduzir a densidade de pontos para cerca de 1 metro de resolução e a partir de cada ponto de Ground clacular a coordenada astronômica (Azimute e ALtura) e a distância linear para cada ponto das classes de Edificação e outras feições.

Para esse primeiro teste vamos apenas usar uma folha SCM, sabendo que inerferências para visão de céu podem se estender muito além dos limites das dimensões de uma folha SCM.

In [297]:
import pdal

In [325]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 5
    }
  ]
}"""

In [326]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [327]:
len(arrays[0])

28449

In [328]:
arrays[0][732]

(333341.33, 7395525.01, 738.06, 26, 1, 1, 1, 0, 6, -14., 17, 6, 356573.27684647)

In [329]:
#dir(r)
r.schema

{'schema': {'dimensions': [{'name': 'X', 'size': 8, 'type': 'floating'},
   {'name': 'Y', 'size': 8, 'type': 'floating'},
   {'name': 'Z', 'size': 8, 'type': 'floating'},
   {'name': 'Intensity', 'size': 2, 'type': 'unsigned'},
   {'name': 'ReturnNumber', 'size': 1, 'type': 'unsigned'},
   {'name': 'NumberOfReturns', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanDirectionFlag', 'size': 1, 'type': 'unsigned'},
   {'name': 'EdgeOfFlightLine', 'size': 1, 'type': 'unsigned'},
   {'name': 'Classification', 'size': 1, 'type': 'unsigned'},
   {'name': 'ScanAngleRank', 'size': 4, 'type': 'floating'},
   {'name': 'UserData', 'size': 1, 'type': 'unsigned'},
   {'name': 'PointSourceId', 'size': 2, 'type': 'unsigned'},
   {'name': 'GpsTime', 'size': 8, 'type': 'floating'}]}}

## Convertendo pontos para uma nd_array

In [330]:
r.arrays[0].dtype

dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])

In [331]:
r.arrays[0]['X'].transpose().shape

(28449,)

In [332]:
import numpy as np

r.arrays[0]['Y']

r.arrays[0][:][0:1]

array([(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])

In [333]:
r.arrays[0]['X']

array([333499.29, 333499.1 , 333499.29, ..., 333520.06, 333587.32,
       333563.24])

## Importando dados para Pandas

In [334]:
import pandas as pd

In [335]:
df = pd.DataFrame(arrays[0])

In [336]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
2,333499.29,7395521.99,751.69,10,1,1,0,0,6,-11.0,17,6,356569.383594
3,333499.27,7395537.84,736.85,9,1,1,0,0,6,-12.0,17,6,356569.383810
4,333499.09,7395543.14,736.53,15,1,1,0,0,6,-13.0,17,6,356569.383898
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28444,333592.36,7395545.25,729.15,3,1,2,1,0,19,7.0,40,6,363043.632359
28445,333554.39,7395545.09,741.76,33,1,1,1,0,6,10.0,40,6,363043.651367
28446,333520.06,7395558.13,729.85,2,1,2,1,0,19,13.0,40,6,363044.033739
28447,333587.32,7395563.87,744.67,14,1,1,1,0,6,7.0,40,6,363044.068740


In [337]:
df.Classification.unique()

array([ 6, 20, 19,  2,  5,  9], dtype=uint8)

## Calculando o Azimute entre dois pontos

In [338]:
import math

def Az(x1,y1,x2,y2):
    deg = math.degrees(math.atan2((x2 - x1),(y2 - y1)))
    if (deg < 0):
        deg += 360.0
    return deg

In [310]:
Az(1, 1, 1, 0)

180.0

## Calculando a Altura ou Elevação do ponto em relação ao Zenite

In [311]:
from scipy.spatial import distance

In [312]:
## Calculando altura
def altura_graus(x1, y1, z1, x2, y2, z2):
    altura = z2 - z1
    distancia = distance.euclidean([x1, y1], [x2, y2])
    return math.degrees(math.atan2(altura, distancia))
    #return distancia

altura_graus(0,0,0,1,1,1)

35.264389682754654

## Gerando pontos x,y,z 

In [339]:
df_ground = pd.DataFrame.from_records(df[df.Classification == 2][['X', 'Y', 'Z']].head(1000))

In [340]:
df_ground

,X,Y,Z
0,333475.91,7395545.19,724.75
1,333471.38,7395557.39,725.11
2,333465.71,7395556.51,725.17
3,333458.28,7395549.13,724.85
4,333454.93,7395553.02,724.84
...,...,...,...
978,333572.35,7395548.48,724.32
979,333589.63,7395543.64,724.11
980,333513.14,7395500.41,724.62
981,333513.53,7395510.17,724.71


In [341]:
df_building = pd.DataFrame.from_records(df[df.Classification == 6][['X', 'Y', 'Z']])
df_building = df_building.rename(columns={'X':'X1', 'Y':'Y1', 'Z':'Z1'})

In [146]:
# %timeit df_building.to_numpy() - df_ground.to_numpy()[0, :]

## Calculando o Azimute com Matriz de pontos

In [342]:
import numpy as np

def np_az(nd_building, nd_ground):
    delta_x = nd_building[:, 0] - nd_ground[0, 0]
    delta_y = nd_building[:, 1] - nd_ground[0, 1]
    degree = np.arctan2(delta_x, delta_y) * 180 / np.pi
    degree[np.less(degree, 0)] += 360.0
    return degree

In [317]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [1.,1.,1.]])
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]])

In [318]:
az = np_az(test_building, test_ground)
#df_building.to_numpy()[:, 0] - df_ground.to_numpy()[0, 0]
az

array([  0.        ,  90.        , 180.        , 270.        ,
       359.98842393])

## Calculando a altura/elevação com matriz de pontos

In [152]:
## Calculando altura/elevação

from scipy.spatial import distance

def np_altura(nd_building, nd_ground):
    altura = nd_building[:, 2] - nd_ground[0, 2]
    distancia = np.sqrt(np.sum((nd_building[:, [0,1]] - nd_ground[0, [0,1]])**2, axis=1))
    return np.arctan2((nd_building[:, 2] - nd_ground[0, 2]), distancia) * 180 / np.pi

In [153]:
altura = np_altura(test_building, test_ground)
altura

array([4.50000000e+01, 4.50000000e+01, 4.50000000e+01, 8.99939005e+01,
       5.78803701e-04])

In [154]:
nd_coordenadas = np.array([az, altura])
nd_coordenadas.transpose().shape

(5, 2)

## Utilizando os dados de Building

In [343]:
nd_building = df_building.to_numpy()
nd_ground = df_ground.to_numpy()

In [148]:
def calc_svf(nd_building, nd_ground):
    az = np_az(nd_building, nd_ground)
    altura = np_altura(nd_building, nd_ground)
    coordenadas = np.array([az, altura])
    return coordenadas.transpose()

In [158]:
#%timeit calc_svf(nd_building, nd_ground)

## Calculando o SVF

Agora para calcular o SVF vamos dividir a semi-esfera visível do céu em N fusos, definido pela variável`quantidade_de_fusos`, e cada um dos fusos terá uma área esférica visível definido pela função `proporcao_visivel_no fuso`.
Depois de disponível essa função vamos iterar ela sobre todos os pontos de ground resultando no retorno da função `proporcao_visivel_total`.

### Refatorando as funções de Azimute e Altura para trabalhar com o vetorização de Ground e Building

Para fins computacionais e menos iterações de laço que tendem a não ser tão eficientes, vamos tentar usar os recursos de vetorização e evitar o laço repetitivo

In [344]:
def np_altura(nd_building, nd_ground):
    n2d_ground_z = nd_ground[:, 2].reshape(-1, 1)
    n2dt_building_z = nd_building[:, 2].reshape(-1, 1).transpose()
    
    nd_altura = np.subtract(n2dt_building_z, n2d_ground_z)
    
    n2d_ground_x = nd_ground[:, 0].reshape(-1, 1)
    n2dt_building_x = nd_building[:, 0].reshape(-1, 1).transpose()
    nd_delta_x = np.subtract(n2dt_building_x, n2d_ground_x)
    
    n2d_ground_y = nd_ground[:, 1].reshape(-1, 1)
    n2dt_building_y = nd_building[:, 1].reshape(-1, 1).transpose()
    nd_delta_y = np.subtract(n2dt_building_y, n2d_ground_y)
    
    nd_distancia = np.hypot(nd_delta_x, nd_delta_y)
    
    nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
    nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0
    
    return nd_distancia

In [345]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [-1.,-1.,-1.], [1.,1.,1.]], dtype='float32')
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]], dtype='float32')
print(test_ground.dtype)
print(test_building.dtype)

float32
float32


In [347]:
%timeit altura = np_altura(nd_building, nd_ground)
altura.shape

962 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


(983, 18835)

In [348]:
len(nd_ground)

983

In [278]:
## Gerando dados de teste
test_ground = np.array([[0.,0.,0.], [-1.,-1.,-1.], [1.,1.,1.]], dtype='float32')
test_building = np.array([[0.,2.,2.], [2.,0,2.], [0.,-2.,2.], [-2.,0,18787.], [-2.,9899.,0.1]], dtype='float32')

In [386]:
def np_altura(nd_building, nd_ground):
    n2d_ground = nd_ground[:, :].reshape(-1, 3).astype('float32')
    n2d_ground = n2d_ground.reshape(-1, 3, 1)

    n2dt_building = nd_building[:, :].reshape(-1, 3).astype('float32')
    n2dt_building = n2dt_building.reshape(-1, 3, 1).transpose()

    nd_delta = np.subtract(n2dt_building, n2d_ground)
    
    nd_altura = nd_delta[:,2,:]
    
    nd_distancia = np.hypot(nd_delta[:,0,:], nd_delta[:,1,:])
    
    nd_elevacao = np.degrees(np.arctan2(nd_altura, nd_distancia))
    nd_elevacao[np.less(nd_elevacao, 0.)] += 360.0
    
    return nd_elevacao

In [388]:
altura = np_altura(nd_building, nd_ground)
altura

array([[19.182251 , 17.367096 , 39.40355  , ..., 12.231013 ,  9.996271 ,
         2.6804206],
       [19.573875 , 18.642534 , 30.482534 , ..., 11.219869 ,  9.561543 ,
         2.4133306],
       [16.423418 , 15.792126 , 28.854101 , ..., 10.512951 ,  9.093957 ,
         2.2383811],
       ...,
       [ 9.297652 ,  8.624779 , 46.630604 , ..., 15.77367  , 11.602692 ,
         2.891229 ],
       [10.686184 ,  9.812995 , 55.375008 , ..., 17.587652 , 12.314791 ,
         3.1114328],
       [13.130521 , 11.876581 , 60.377926 , ..., 20.287586 , 13.348857 ,
         3.542544 ]], dtype=float32)

In [371]:
altura.reshape(983, 1, 18835)

array([[[19.182251 , 17.367096 , 39.40355  , ..., 12.231013 ,
          9.996271 ,  2.6804206]],

       [[19.573875 , 18.642534 , 30.482534 , ..., 11.219869 ,
          9.561543 ,  2.4133306]],

       [[16.423418 , 15.792126 , 28.854101 , ..., 10.512951 ,
          9.093957 ,  2.2383811]],

       ...,

       [[ 9.297652 ,  8.624779 , 46.630604 , ..., 15.77367  ,
         11.602692 ,  2.891229 ]],

       [[10.686184 ,  9.812995 , 55.375008 , ..., 17.587652 ,
         12.314791 ,  3.1114328]],

       [[13.130521 , 11.876581 , 60.377926 , ..., 20.287586 ,
         13.348857 ,  3.542544 ]]], dtype=float32)

In [81]:
import numpy as np
## Testando outer
a = np.array([[1,2,3,4,5,6]])
b = a.transpose()
c = np.subtract(a, b)
c.shape

(6, 6)